In [1]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from models import Uni_Sign
import utils as utils
from datasets import S2T_Dataset
import os
import time
import argparse, json, datetime
from pathlib import Path
import math
import sys
from timm.optim import create_optimizer
from models import get_requires_grad_dict
from SLRT_metrics import translation_performance, islr_performance, wer_list
from transformers import get_scheduler
from config import *
import gzip, pickle, os


/home/shenchris/miniconda3/envs/Uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-11-28 05:09:49,067] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/shenchris/miniconda3/envs/Uni/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/shenchris/miniconda3/envs/Uni/lib/python3.9/site-packages/torch/utils/cpp_extension.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]
/home/shenchris/miniconda3/envs/Uni/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [ ]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

# 手动创建 args 对象
args = Args(
    batch_size=8,
    gradient_accumulation_steps=1,
    epochs=20,
    opt='AdamW',
    lr=3e-4,
    output_dir='out/test',
    finetune='out/wlasl_rgb_pose_islr.pth',
    dataset='WLASL',
    task='ISLR',
    eval=True,
    rgb_support=True,
    max_length = 100
    # 添加 fine_tuning.py 或 utils.py 中所有需要的参数
)

In [ ]:
train_data = S2T_Dataset(path=train_label_paths[args.dataset], 
                             args=args, phase='train')
dev_data = S2T_Dataset(path=dev_label_paths[args.dataset], 
                             args=args, phase='dev')
test_data = S2T_Dataset(path=test_label_paths[args.dataset], 
                             args=args, phase='test')

In [13]:
path = "data/WLASL/labels-2000.train"
# load
with gzip.open(path, "rb") as f:
    train_data = pickle.load(f) 

path = "data/WLASL/labels-2000.dev"
# load
with gzip.open(path, "rb") as f:
    dev_data = pickle.load(f)

path = "data/WLASL/labels-2000.test"
# load
with gzip.open(path, "rb") as f:
    test_data = pickle.load(f)

In [ ]:
# train_data

In [ ]:
# print(len(train_data.list))
# print(len(dev_data.list))
# print(len(test_data.list))

In [14]:

GLOSS = [
    "book","drink","computer","before","chair","go","clothes","who","candy","cousin","deaf","fine","help","no","thin","walk","year","yes","all","black","cool","finish","hot","like","many","mother","now","orange","table","thanksgiving","what","woman","bed","blue","bowling","can","dog","family","fish","graduate","hat","hearing","kiss","language","later","man","shirt","study","tall","white","wrong","accident","apple","bird","change","color","corn","cow","dance","dark","doctor","eat","enjoy","forget","give","last","meet","pink","pizza","play","school","secretary","short","time","want","work","africa","basketball","birthday","brown","but","cheat","city","cook","decide","full","how","jacket","letter","medicine","need","paint","paper","pull","purple","right","same","son","tell","thursday",
]

In [ ]:
# for key, val in train_data.raw_data.items():
#     print(key)
#     break

In [ ]:
# train_new_raw_data = {
#     key: val 
#     for key, val in train_data.raw_data.items() 
#     if val['text'] in GLOSS
# }
# dev_new_raw_data = {
#     key: val 
#     for key, val in dev_data.raw_data.items() 
#     if val['text'] in GLOSS
# }
# test_new_raw_data = {
#     key: val 
#     for key, val in test_data.raw_data.items() 
#     if val['text'] in GLOSS
# }

In [ ]:
# train_new_list = [
#     key 
#     for key in train_data.list 
#     if key in train_new_raw_data
# ]
# dev_new_list = [
#     key 
#     for key in dev_data.list 
#     if key in dev_new_raw_data
# ]
# test_new_list = [
#     key 
#     for key in test_data.list 
#     if key in test_new_raw_data
# ]

In [ ]:
# # 4. Overwrite the dataset attributes
# train_data.raw_data = train_new_raw_data
# train_data.list = train_new_list
# dev_data.raw_data = dev_new_raw_data
# dev_data.list = dev_new_list
# test_data.raw_data = test_new_raw_data
# test_data.list = test_new_list
# # --- Verification ---
# print(f"Filtered Dataset Size: {len(train_data)}")
# print(f"Dictionary Size: {len(train_data.raw_data)}")
# print(f"List Size: {len(train_data.list)}")
# print(f"Filtered Dataset Size: {len(dev_data)}")
# print(f"Dictionary Size: {len(dev_data.raw_data)}")
# print(f"List Size: {len(dev_data.list)}")
# print(f"Filtered Dataset Size: {len(test_data)}")
# print(f"Dictionary Size: {len(test_data.raw_data)}")
# print(f"List Size: {len(test_data.list)}")

In [4]:
type(train_data)

dict

In [15]:
train_new_raw_data = {k: v for k, v in train_data.items() if v.get("text") in GLOSS}
dev_new_raw_data = {k: v for k, v in dev_data.items() if v.get("text") in GLOSS}
test_new_raw_data = {k: v for k, v in test_data.items() if v.get("text") in GLOSS}


In [16]:
print(len(train_new_raw_data))

1442


In [10]:
train_new_raw_data['00001'] = {'name': '00001', 'gloss': '', 'text': '', 'video_path': '00001.mp4'}
train_new_raw_data['00002'] = {'name': '00002', 'gloss': '', 'text': '', 'video_path': '00002.mp4'}
train_new_raw_data['00003'] = {'name': '00003', 'gloss': '', 'text': '', 'video_path': '00003.mp4'}
train_new_raw_data['00004'] = {'name': '00004', 'gloss': '', 'text': '', 'video_path': '00004.mp4'}
train_new_raw_data['00005'] = {'name': '00005', 'gloss': '', 'text': '', 'video_path': '00005.mp4'}
train_new_raw_data['00006'] = {'name': '00006', 'gloss': '', 'text': '', 'video_path': '00006.mp4'}

In [11]:
out_path = "data/WLASL/labels-100.train"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
with gzip.open(out_path, "wb") as f:
    pickle.dump(train_new_raw_data, f)
print(f"saved {len(train_new_raw_data)} samples to {out_path}")

saved 1448 samples to data/WLASL/labels-100.train


In [105]:
out_path = "data/WLASL/labels-100.dev"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
with gzip.open(out_path, "wb") as f:
    pickle.dump(dev_new_raw_data, f)
print(f"saved {len(dev_new_raw_data)} samples to {out_path}")

saved 338 samples to data/WLASL/labels-100.dev


In [106]:
out_path = "data/WLASL/labels-100.test"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
with gzip.open(out_path, "wb") as f:
    pickle.dump(test_new_raw_data, f)
print(f"saved {len(test_new_raw_data)} samples to {out_path}")

saved 258 samples to data/WLASL/labels-100.test
